In [62]:
### basic libraries
import os
from json import dumps
import pandas as pd 

### carto libraries
import shapefile
import geopandas

from app.scripts.topojson import topojson

from pyproj import Proj, transform
inCRS  = 'epsg:2154'
outCRS = 'epsg:4326'
inProj  = Proj(init=inCRS) # proj in  : Lambert 93
outProj = Proj(init=outCRS) # proj out : WSG 84

### TO DO : find topojson libraries to reduce output file size


In [19]:
### basic folders addresses and names
cwd = os.getcwd()

data_folder = "app/static/data"
carto_folder = "carto"

carto_path = os.path.join(cwd, data_folder, carto_folder)

print "-- cwd :", cwd
print "-- carto path : ", carto_path


-- cwd : /Users/jpy/Dropbox/_FLASK/concours_pesticides
-- carto path :  /Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/carto


In [58]:

def checkDTypes (df) :
    # check data type
    
    for index in df.index.names :
        print "---- index : ", index

    for col in df.columns :
        #label = col.values
        dtype = df[col].dtype
        
        print "---- dtypes col : ", col, "/", dtype
        

In [20]:
# cf : https://pypi.python.org/pypi/pyshp
# cf : http://gis.stackexchange.com/questions/183795/how-do-i-select-shapefiles-to-be-converted-to-geojson-in-folder-with-multiple-sh
# cf : https://github.com/mlaloux/Python-geo_interface-applications/blob/master/PyShp_geointerface.py

# cf : https://glenbambrick.com/tag/pyshp/
# cf : https://glenbambrick.com/2016/01/24/reproject-shapefile/

In [21]:
### SHP files 
_shp  = ".shp"
_json = ".json"
_copy = "_copy"

water_shp_fname = "PolygMasseDEauSouterraine"

## TO DO : add departements .shp 

In [22]:
### reading/writing - converting shp files to geojson
    
def geofile_path(filename, extension):
    path = os.path.join(carto_path , filename + extension )
    print path
    return path


In [69]:
'''
def readSHP(filename):  
    # generator 
    reader = shapefile.Reader( root_carto_folder+ filename + _shp )  
    fields = reader.fields[1:]  
    field_names = [field[0] for field in fields]  
    for sr in reader.shapeRecords():  
        geom = sr.shape.__geo_interface__  
        atr = dict(zip(field_names, sr.record))  
        yield dict(geometry=geom,properties=atr)    
'''

def readSHP(filename) :
    
    # read the shapefile
    reader = shapefile.Reader( geofile_path( water_shp_fname, _shp ) )
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]

    buffer = []
    for sr in reader.shapeRecords():
       atr = dict(zip(field_names, sr.record))
       geom = sr.shape.__geo_interface__
       buffer.append(dict(type="Feature", geometry=geom, properties=atr)) 

    # write the GeoJSON file (copy)
    geojson = open( os.path.join( carto_path, filename + _copy + _json), "w")
    geojson.write(dumps({"type": "FeatureCollection", "features": buffer}, indent=2) + "\n")
    geojson.close()
    

In [70]:
### options for gdf.read_file()
#import fiona; help(fiona.open)

### options for gdf.to_file()
#import fiona; fiona.supported_drivers

In [86]:
### read .shp with geopandas

shp_encoding = "utf-8"

gdf = geopandas.read_file( geofile_path( water_shp_fname, _shp ), crs=None, encoding=shp_encoding )

gdf.set_index(["CdBassinDC", "CdMasseDEa"], inplace=True)
gdf.sortlevel(inplace=True) 


/Users/jpy/Dropbox/_FLASK/concours_pesticides/app/static/data/carto/PolygMasseDEauSouterraine.shp


In [87]:
gdf.crs

{u'ellps': u'GRS80',
 u'lat_0': 46.5,
 u'lat_1': 49,
 u'lat_2': 44,
 u'lon_0': 3,
 u'no_defs': True,
 u'proj': u'lcc',
 u'units': u'm',
 u'x_0': 700000,
 u'y_0': 6600000}

In [84]:
gdf.head()


CdEcoregio CdEuMasseD CdPolygMas Commentair  DateCreati  \
CdBassinDC CdMasseDEa                                                           
A          AG001              13    FRAG001   00000002       None  31/12/2004   
           AG001              13    FRAG001   00000001       None  31/12/2004   
           AG002              13    FRAG002   00000003       None  31/12/2004   
           AG003              13    FRAG003   00000005       None  31/12/2004   
           AG003              13    FRAG003   00000004       None  31/12/2004   

                       DateMajMas FrangeLitt Karstique    LatMasseDE  \
CdBassinDC CdMasseDEa                                                  
A          AG001       18/12/2013          N         N  7.069081e+06   
           AG001       18/12/2013          N         N  7.069081e+06   
           AG002       18/12/2013          N         Y  7.072421e+06   
           AG003       18/12/2013          N         N  7.038903e+06   
           AG003       18/12/2013          N         N  7.038903e+06   

                          LonMasseDE  \
CdBassinDC CdMasseDEa                  
A          AG001       639573.042158   
           AG001       639573.042158   
           AG002       609585.385827   
           AG003       685405.450775   
           AG003       685405.450775   

                                             ...                          \
CdBassinDC CdMasseDEa                        ...                           
A          AG001                             ...                           
           AG001                             ...                           
           AG002                             ...                           
           AG003                             ...                           
           AG003                             ...                           

                                             NomMasseDE PrecSupMas Regroupees  \
CdBassinDC CdMasseDEa                                                           
A          AG001                Craie de lâAudomarois          1          N   
           AG001                Craie de lâAudomarois          1          N   
           AG002                Calcaires du Boulonnais          2          Y   
           AG003       Craie de la vallÃ©e de la DeÃ»le          1          N   
           AG003       Craie de la vallÃ©e de la DeÃ»le          1          N   

                       StMasseDEa SurfaceAff SurfaceSsC SurfaceTot SystemeRef  \
CdBassinDC CdMasseDEa                                                           
A          AG001          ValidÃ©      868.2       82.4      950.6         26   
           AG001          ValidÃ©      868.2       82.4      950.6         26   
           AG002          ValidÃ©      477.4        0.0      477.4         26   
           AG003          ValidÃ©      742.4      587.5     1329.9         26   
           AG003          ValidÃ©      742.4      587.5     1329.9         26   

                       TypeMasseD  \
CdBassinDC CdMasseDEa               
A          AG001               DS   
           AG001               DS   
           AG002               DS   
           AG003               DS   
           AG003               DS   

                                                                geometry  
CdBassinDC CdMasseDEa                                                     
A          AG001       POLYGON ((651859.6700888127 7067573.305585384,...  
           AG001       (POLYGON ((617227.0852775276 7096815.711801797...  
           AG002       POLYGON ((617970.4125330001 7063544.312811732,...  
           AG003       (POLYGON ((719164.1102770045 7051492.354207844...  
           AG003       POLYGON ((704207.8221555278 7065745.600930788,...  

[5 rows x 24 columns]

In [85]:
def colToDate(df, list_col_names):
    # change to date format
    
    for col in list_col_names :
        df.loc[:, col] = pd.to_datetime( df.loc[:, col], infer_datetime_format=True)
    
    return df

# change date format
gdf = colToDate( gdf, ["DateCreati", "DateMajMas"] )

checkDTypes (gdf) 

gdf.head()


---- index :  CdBassinDC
---- index :  CdMasseDEa
---- dtypes col :  CdEcoregio / object
---- dtypes col :  CdEuMasseD / object
---- dtypes col :  CdPolygMas / object
---- dtypes col :  Commentair / object
---- dtypes col :  DateCreati / datetime64[ns]
---- dtypes col :  DateMajMas / datetime64[ns]
---- dtypes col :  FrangeLitt / object
---- dtypes col :  Karstique / object
---- dtypes col :  LatMasseDE / float64
---- dtypes col :  LonMasseDE / float64
---- dtypes col :  MasseDEauA / object
---- dtypes col :  MasseDEauT / object
---- dtypes col :  NatureEcou / object
---- dtypes col :  Niveau / int64
---- dtypes col :  NomMasseDE / object
---- dtypes col :  PrecSupMas / object
---- dtypes col :  Regroupees / object
---- dtypes col :  StMasseDEa / object
---- dtypes col :  SurfaceAff / float64
---- dtypes col :  SurfaceSsC / float64
---- dtypes col :  SurfaceTot / float64
---- dtypes col :  SystemeRef / object
---- dtypes col :  TypeMasseD / object
---- dtypes col :  geometry / object


CdEcoregio CdEuMasseD CdPolygMas Commentair DateCreati  \
CdBassinDC CdMasseDEa                                                          
A          AG001              13    FRAG001   00000002       None 2004-12-31   
           AG001              13    FRAG001   00000001       None 2004-12-31   
           AG002              13    FRAG002   00000003       None 2004-12-31   
           AG003              13    FRAG003   00000005       None 2004-12-31   
           AG003              13    FRAG003   00000004       None 2004-12-31   

                      DateMajMas FrangeLitt Karstique    LatMasseDE  \
CdBassinDC CdMasseDEa                                                 
A          AG001      2013-12-18          N         N  7.069081e+06   
           AG001      2013-12-18          N         N  7.069081e+06   
           AG002      2013-12-18          N         Y  7.072421e+06   
           AG003      2013-12-18          N         N  7.038903e+06   
           AG003      2013-12-18          N         N  7.038903e+06   

                          LonMasseDE  \
CdBassinDC CdMasseDEa                  
A          AG001       639573.042158   
           AG001       639573.042158   
           AG002       609585.385827   
           AG003       685405.450775   
           AG003       685405.450775   

                                             ...                          \
CdBassinDC CdMasseDEa                        ...                           
A          AG001                             ...                           
           AG001                             ...                           
           AG002                             ...                           
           AG003                             ...                           
           AG003                             ...                           

                                             NomMasseDE PrecSupMas Regroupees  \
CdBassinDC CdMasseDEa                                                           
A          AG001                Craie de lâAudomarois          1          N   
           AG001                Craie de lâAudomarois          1          N   
           AG002                Calcaires du Boulonnais          2          Y   
           AG003       Craie de la vallÃ©e de la DeÃ»le          1          N   
           AG003       Craie de la vallÃ©e de la DeÃ»le          1          N   

                       StMasseDEa SurfaceAff SurfaceSsC SurfaceTot SystemeRef  \
CdBassinDC CdMasseDEa                                                           
A          AG001          ValidÃ©      868.2       82.4      950.6         26   
           AG001          ValidÃ©      868.2       82.4      950.6         26   
           AG002          ValidÃ©      477.4        0.0      477.4         26   
           AG003          ValidÃ©      742.4      587.5     1329.9         26   
           AG003          ValidÃ©      742.4      587.5     1329.9         26   

                       TypeMasseD  \
CdBassinDC CdMasseDEa               
A          AG001               DS   
           AG001               DS   
           AG002               DS   
           AG003               DS   
           AG003               DS   

                                                                geometry  
CdBassinDC CdMasseDEa                                                     
A          AG001       POLYGON ((651859.6700888127 7067573.305585384,...  
           AG001       (POLYGON ((617227.0852775276 7096815.711801797...  
           AG002       POLYGON ((617970.4125330001 7063544.312811732,...  
           AG003       (POLYGON ((719164.1102770045 7051492.354207844...  
           AG003       POLYGON ((704207.8221555278 7065745.600930788,...  

[5 rows x 24 columns]

In [79]:
gdf.loc["H","HG210"]


CdEcoregio CdEuMasseD CdPolygMas Commentair DateCreati  \
CdBassinDC CdMasseDEa                                                          
H          HG210              13    FRHG210   00001100       None 2004-12-31   
           HG210              13    FRHG210   00001028       None 2004-12-31   
           HG210              13    FRHG210   00001029       None 2004-12-31   

                      DateMajMas FrangeLitt Karstique    LatMasseDE  \
CdBassinDC CdMasseDEa                                                 
H          HG210      2013-12-18          N         Y  6.762451e+06   
           HG210      2013-12-18          N         Y  6.762451e+06   
           HG210      2013-12-18          N         Y  6.762451e+06   

                          LonMasseDE  \
CdBassinDC CdMasseDEa                  
H          HG210       701434.579909   
           HG210       701434.579909   
           HG210       701434.579909   

                                             ...                          \
CdBassinDC CdMasseDEa                        ...                           
H          HG210                             ...                           
           HG210                             ...                           
           HG210                             ...                           

                               NomMasseDE PrecSupMas Regroupees  StMasseDEa  \
CdBassinDC CdMasseDEa                                                         
H          HG210       Craie du GÃ¢tinais          1          Y     ValidÃ©   
           HG210       Craie du GÃ¢tinais          1          Y     ValidÃ©   
           HG210       Craie du GÃ¢tinais          1          Y     ValidÃ©   

                      SurfaceAff SurfaceSsC SurfaceTot SystemeRef  TypeMasseD  \
CdBassinDC CdMasseDEa                                                           
H          HG210          3565.5       57.5     3623.0         26          DS   
           HG210          3565.5       57.5     3623.0         26          DS   
           HG210          3565.5       57.5     3623.0         26          DS   

                                                                geometry  
CdBassinDC CdMasseDEa                                                     
H          HG210       POLYGON ((681077.108196184 6722325.508935641, ...  
           HG210       (POLYGON ((681683.8945573717 6721773.287093535...  
           HG210       (POLYGON ((681217.2994322404 6722139.408087388...  

[3 rows x 24 columns]

In [18]:
### test read/write /// .shp --> .json

#readSHP(water_shp_fname) ### problem : 1.16 Go file !!!